In [18]:
# Carica il file csv e converto colonna dei prezzi in int
import pandas as pd
import re

df=pd.read_csv("roma_dati_booking.csv")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df.columns

Index(['nome_hotel', 'prezzo', 'stanza', 'città', 'data', 'punteggio',
       'numero_recensioni', 'distanza_centro', 'genius', 'offerte',
       'colazione_inclusa', 'senza_pagamento_anticipato',
       'cancellazione_gratuita', 'os', 'username', 'user_agent_type',
       'TIMESTAMP'],
      dtype='object')

In [31]:
# Crea delle colonne con valori True/False in base alle offerte
df['booking_paga'] = df['offerte'].str.contains('Booking.com paga')
df['offerta_tempo_limitato'] = df['offerte'].str.contains('tempo limitato')
df['offerta_solo_mobile'] = df['offerte'].str.contains('mobile')
df['offerta_super_segreta'] = df['offerte'].str.contains('SuperSegreta')
df['offerta_inizio_2024'] = df['offerte'].str.contains('Offerta Inizio 2024')
# Inserisce False nei campi nulli
df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']] = df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']].fillna(False)

In [20]:
# Visualizza lista ricerche in base ad orari
df['TIMESTAMP'].unique()

array(['03-03-2024---13:49', '03-03-2024---14:02', '03-03-2024---14:16',
       '03-03-2024---14:31', '03-03-2024---15:01', '03-03-2024---15:12',
       '03-03-2024---15:22', '03-03-2024---15:37', '03-03-2024---16:00',
       '03-03-2024---16:44'], dtype=object)

In [25]:
# Divide il dataframe in base a orari ricerche per fare poi il merge
orario1 = df["TIMESTAMP"].unique()[-1]
orario2 = df["TIMESTAMP"].unique()[-4]

df1 = df[df["TIMESTAMP"]==orario1]
df2 = df[df["TIMESTAMP"]==orario2]

# Eliminazione duplicati
df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()

In [26]:
# MERGE TRA DATAFRAME
'''
Se le ricerche sono dello stesso tipo (Mobile o Desktop) il merge avviene anche sulla colonna stanza
Vengono inoltre compattate eventuali colonne uguali e prese in considerazione le righe in cui vi è una differenza di prezzo
'''
df_list = df.columns.tolist()
df_list =  df_list[3:4] +df_list[5:-2]
if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data'], how='inner')
else:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','data'], how='inner')
merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
for col in df_list:
    if merged_df[col+"_x"].equals(merged_df[col+"_y"]):
        merged_df[col] = merged_df[col+"_x"]
        merged_df.drop(columns=[col+"_x",col+"_y"],inplace=True)
# Elimino hotel con stesso nome
merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)        
merged_df.columns

Index(['nome_hotel', 'prezzo_x', 'stanza_x', 'città_x', 'data', 'punteggio_x',
       'numero_recensioni_x', 'distanza_centro_x', 'offerte_x',
       'colazione_inclusa_x', 'senza_pagamento_anticipato_x',
       'cancellazione_gratuita_x', 'os_x', 'username_x', 'user_agent_type_x',
       'TIMESTAMP_x', 'booking_paga_x', 'offerta_tempo_limitato_x',
       'prezzo_mobile_x', 'offerta_super_segreta_x', 'offerta_inizio_2024_x',
       'prezzo_y', 'stanza_y', 'città_y', 'punteggio_y', 'numero_recensioni_y',
       'distanza_centro_y', 'offerte_y', 'colazione_inclusa_y',
       'senza_pagamento_anticipato_y', 'cancellazione_gratuita_y', 'os_y',
       'username_y', 'user_agent_type_y', 'TIMESTAMP_y', 'booking_paga_y',
       'offerta_tempo_limitato_y', 'prezzo_mobile_y',
       'offerta_super_segreta_y', 'offerta_inizio_2024_y', 'genius'],
      dtype='object')

In [29]:
# Per visualizzare tutte le righe con differenze di prezzo
#pd.set_option('display.max_rows', None)
merged_df[["nome_hotel","prezzo_x","prezzo_y","user_agent_type_x","user_agent_type_y",'os_x','os_y']]


,nome_hotel,prezzo_x,prezzo_y,user_agent_type_x,user_agent_type_y,os_x,os_y
0,The Place Giuliana,85,79,desktop,mobile,Windows,Android
3,Hotel Sant'Angelo,96,148,desktop,mobile,Windows,Android
4,Borgognona Rooms,152,141,desktop,mobile,Windows,Android
5,Love of Rome,69,70,desktop,mobile,Windows,Android
6,Gardenia Suite,116,119,desktop,mobile,Windows,Android
7,Hotel Millerose,111,113,desktop,mobile,Windows,Android
8,Frattina FF italian suites,152,155,desktop,mobile,Windows,Android
9,A Peace Of Rome,140,130,desktop,mobile,Windows,Android
10,Corso Boutique Luxury Rooms,181,185,desktop,mobile,Windows,Android
11,LHG Comfy Rooms,101,111,desktop,mobile,Windows,Android


In [ ]:
#aggiungo colonna con differenza prezzi
pd.set_option('display.max_rows', None)
merged_df["differenza_prezzo"] = merged_df["prezzo_x"]-merged_df["prezzo_y"]
merged_df.sort_values(by='differenza_prezzo',ascending= False)
merged_df[['nome_hotel','prezzo_x','prezzo_y','differenza_prezzo','offerte_x','offerte_y']]



In [24]:
# Visualizza dove compare una specifica offerta in almeno una delle ricerche
df_mobile = merged_df[(merged_df['offerta_super_segreta_x'])|(merged_df['offerta_super_segreta_y'])]
df_mobile[['nome_hotel','offerta_super_segreta_x','offerta_super_segreta_y','prezzo_x','prezzo_y','offerte_x','offerte_y','os','username_x','username_y']]


,nome_hotel,offerta_super_segreta_x,offerta_super_segreta_y,prezzo_x,prezzo_y,offerte_x,offerte_y,os,username_x,username_y
41,Babuino Palace&Suites,True,True,185,171,Offerta SuperSegreta,Offerta SuperSegreta Booking.com paga,Android,marcofantile,pantilaura56
171,Tree Charme Parliament Boutique Hotel,True,True,264,244,Offerta SuperSegreta,Offerta SuperSegreta Booking.com paga,Android,marcofantile,pantilaura56
200,BQ House Castello Luxury Rooms,True,False,181,172,Prezzo solo su mobile Offerta SuperSegreta,Offerta Inizio 2024 Booking.com paga,Android,marcofantile,pantilaura56
239,Hotel Via Veneto,True,True,349,323,Prezzo solo su mobile Offerta SuperSegreta,Offerta SuperSegreta Booking.com paga,Android,marcofantile,pantilaura56
249,Prati Boutique Suites,True,True,98,92,Offerta SuperSegreta,Prezzo solo su mobile Offerta SuperSegreta Boo...,Android,marcofantile,pantilaura56
270,La Casa di Emanuela,True,True,494,470,Offerta SuperSegreta,Offerta SuperSegreta Booking.com paga,Android,marcofantile,pantilaura56
472,Hotel Giolli Nazionale,True,True,269,248,Prezzo solo su mobile Offerta SuperSegreta,Prezzo solo su mobile Offerta SuperSegreta Boo...,Android,marcofantile,pantilaura56
630,Vibe Nazionale,True,True,269,248,Prezzo solo su mobile Offerta SuperSegreta,Prezzo solo su mobile Offerta SuperSegreta Boo...,Android,marcofantile,pantilaura56
658,Navona Queen Rooftop,True,True,200,185,Prezzo solo su mobile Offerta SuperSegreta,Prezzo solo su mobile Offerta SuperSegreta Boo...,Android,marcofantile,pantilaura56
704,Atypical Rooms,False,True,443,391,Offerta Inizio 2024,Prezzo solo su mobile Offerta SuperSegreta Boo...,Android,marcofantile,pantilaura56


In [ ]:
# Elimino righe in cui il mobile non ha la stanza nel confronto mobile/Desktop
if merged_df['user_agent_type_x'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_x'].dropna()
elif merged_df['user_agent_type_y'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_y'].dropna()
len(df_senza_stanza)